Load Dataset

Dataset is created in the 'create_dataset.ipynb' notebook

In [18]:
import pandas as pd

In [19]:
train_path = r'training_samples.xlsx'
test_path = r'testing_samples.xlsx'

train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [20]:
train_df.head()

,Unnamed: 0,prompt,response
0,0,You are the CEO of a tech startup. Your compan...,"Given the situation, it's crucial to consider ..."
1,1,You are a city mayor and your city has been ex...,Addressing a rising crime rate requires a bala...
2,2,You are a school principal and your school's s...,Addressing low standardized test scores requir...
3,3,You are the manager of a manufacturing plant a...,Addressing production delays requires a carefu...
4,4,You are the director of a hospital and you're ...,Addressing the issue of an overwhelmed hospita...


#### Install necessary libraries

In [21]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [22]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#### Define Hyperparameters

In [23]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "train.jsonl"
finetuned_model_name = "llama-2-7b-lora"

# LoRA configuration
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# 4 bit quantization parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

num_train_epochs = 5
fp16 = False
bf16 = False

per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True

max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"

max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
output_dir = "./results"   # save tensorboard logs
device_map = {"": 0}

#### Load Datasets and Train

In [24]:
# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
# 'system_message' is created from the 'create_dataset.ipynb' notebook
system_message = "Given the complex scenario and multiple factors, provide a well-reasoned recommendation or action plan."

In [26]:
# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [27]:
'''
4-bit quantization with 16-bit floating point precision for computations (training)

Configuration Used
- Using 4-bit Quantization (use_4bit = True) : This setting activates the 4-bit quantization, meaning the model's weights are stored using only 4 bits. 

- Compute Precision (bnb_4bit_compute_dtype = "float16"): Even though the model weights are quantized to 4 bits, computations (inference/training) 
                                                          are carried out using 16-bit floating point precision. This is a compromise between the 
                                                          computational efficiency of lower bit widths and the numerical stability and precision offered by higher bit widths.

- Quantization Type (bnb_4bit_quant_type = "nf4") : Normal Float 4

- Nested Quantization (use_nested_quant = False) : This option, when set to 'True', allow for a more complex quantization scheme that might involve multiple stages or layers of quantization. 
                                                    Setting this to 'False' simplifies the process, focusing on straightforward quantization without additional layers.

https://huggingface.co/docs/transformers/main/en/quantization
'''
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [28]:
# 1: Load Model from huggingface and save it in the local folder
model = AutoModelForCausalLM.from_pretrained(model_name)
model.save_pretrained('llama-2-7b-raw')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# 2: Model Quantization
model = AutoModelForCausalLM.from_pretrained(
    'llama-2-7b-raw',
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [31]:
'''
### LoRA (Low-Rank Adaptation) Parameters

1. lora_r = 64: rank of the low-rank matrices to 64.
            The rank (r) determines the size of these matrices. A higher rank means a more expressive model (capable of capturing more complex modifications) 
            but also increases computational requirements.

2. lora_alpha = 16: This parameter controls the learning rate multiplier for the LoRA parameters. 
                    It adjusts how much the LoRA parameters should learn relative to the base model parameters during adaptation.

3. lora_dropout = 0.1:  Dropout is a regularization technique used to prevent overfitting

'''
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [32]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# 3: Train the model 
trainer.train()
trainer.model.save_pretrained(finetuned_model_name)

/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, 

Step,Training Loss,Validation Loss
5,1.801000,1.585505
10,1.406100,1.330665


In [33]:
# 4: Test the model
instruction_prompt = test_df['prompt'][0]
logging.set_verbosity(logging.CRITICAL)

# system_message followed by instruction
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{instruction_prompt}. [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Given the complex scenario and multiple factors, provide a well-reasoned recommendation or action plan.
<</SYS>>

You are the mayor of a city that is struggling with traffic congestion. You have two options: invest in improving public transportation or implement congestion charging. What is your decision?. [/INST]  As the mayor of a city struggling with traffic congestion, I understand the importance of addressing this issue to improve the quality of life for our citizens and the overall economic health of the city. After careful consideration of the two options presented, I would recommend implementing congestion charging.

While improving public transportation is a worthwhile investment, it may not be the most effective solution to address traffic congestion in the short term. Public transportation systems require significant investment and time to develop, and it may take years for the benefits to be realized


In [34]:
# Test the model for test_df
for index, row in test_df.iterrows():
    instruction_prompt =  row['prompt']

    prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{instruction_prompt}[/INST]"
    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=1500)
    result = gen(prompt)
    print(f"=== {index} ===")
    print(f'{prompt}\n')
    print(result[0]['generated_text'].replace(prompt, ''))

=== 0 ===
[INST] <<SYS>>
Given the complex scenario and multiple factors, provide a well-reasoned recommendation or action plan.
<</SYS>>

You are the mayor of a city that is struggling with traffic congestion. You have two options: invest in improving public transportation or implement congestion charging. What is your decision?[/INST]

  As the mayor of a city struggling with traffic congestion, I understand the importance of addressing this issue to improve the quality of life for our citizens and the overall economic health of our city. After careful consideration of the two options presented, I would recommend implementing congestion charging.

While improving public transportation is a worthwhile investment, it may not be the most effective solution to address traffic congestion in the short term. Public transportation systems often require significant time and resources to develop and expand, and it may take years for the benefits of improved public transportation to be felt thr

### Run Inference

In [35]:
from transformers import pipeline

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nWrite a function that reverses a string. [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 500  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

  Sure! Here is a function that reverses a string in JavaScript:
```
function reverseString(str) {
  return str.split("").reverse().join("");
}
```
Explanation:

* `str.split("")` splits the string into an array of individual characters.
* `reverse()` reverses the order of the array.
* `join("")` joins the array back into a string.

For example, if we call the function with the string "hello", it will return "lohel".

Alternatively, you could use a simple loop to reverse the string:
```
function reverseString(str) {
  var rev = "";
  for (var i = str.length - 1; i >= 0; i--) {
    rev += str[i];
  }
  return rev;
}
```
This function works in the same way as the first one, but it uses a loop to iterate over the characters of the string and concatenate them into a new string in reverse order.


#### Merge the model and store

In [36]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
model_path = "llama-2-7b-finetuned"  # change to your preferred path

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('llama-2-7b-finetuned/tokenizer_config.json',
 'llama-2-7b-finetuned/special_tokens_map.json',
 'llama-2-7b-finetuned/tokenizer.json')

#### Load a fine-tuned model and run inference

In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "llama-2-7b-finetuned" 

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
from transformers import pipeline

prompt = "What is 2 + 2?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = gen(prompt)
print(result[0]['generated_text'])

/home/teaai/anaconda3/envs/tishan_env/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What is 2 + 2?

Answer: 2 + 2 = 4


In [40]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Apr 28 09:20:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 